# Lab 3 - Model training and evaluation
In this lab we will train a multinomial classification model using the bottleneck features created in Lab 2.


![Transfer Learning](https://github.com/jakazmie/images-for-hands-on-labs/raw/master/tlcl.png)

We will start with training a simple logistic regression model in a local environment to validate that the bottleneck features can improve our classifier. We will then use the Azure ML feature called `Automated ML` to find the most optimal model for our image classification task.

![AML Arch](https://github.com/jakazmie/images-for-hands-on-labs/raw/master/automated-machine-learning.png)



## Train logistic regression model locally

We will start by training a logistic regression model locally using the bottleneck features created in Lab 2.

### Connect to AML workspace


In [1]:
import azureml.core
from azureml.core import Workspace
import os

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

Found the config file in: /Users/jarekk/repos/jakazmie/MTC_AzureAILabs/DataScienceTrack/01-aml-walkthrough/aml_config/config.json
jkamllab
jkamllab
eastus2
952a710c-8d9c-40c1-9fec-f752138cc0b3


### Download bottleneck features

In the previous lab, the bottleneck features have been uploaded to the `bottleneck_features`folder in the default datastore.

In [2]:
data_folder = 'bottleneck_features'

In [3]:
ds = ws.get_default_datastore()
ds.download(target_path='.', prefix=data_folder, overwrite=True)
!ls bottleneck_features

aerial_bottleneck_resnet50.h5


### Train logistic regression using bottleneck features

In [4]:
import h5py
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


# Load features and labels into numpy arrays
file_name = os.path.join(data_folder, 'aerial_bottleneck_resnet50.h5')
with h5py.File(file_name, "r") as hfile:
    features = np.array(hfile.get('features'))
    labels = np.array(hfile.get('labels'))
 
# Split the data into training and validation partitions   
X_train, X_validation, y_train, y_validation = train_test_split(features, labels,
                                                               test_size=0.1,
                                                               shuffle=True,
                                                               stratify=labels)
    
# Train logistics regresssion model
reg = 0.1
clf = LogisticRegression(
        C=1.0/reg, 
        multi_class='multinomial',
        solver='lbfgs',
        random_state=1)

clf.fit(X_train, y_train)

LogisticRegression(C=10.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=1, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

### Evaluate the logistic regression model

In [5]:
# Validate
y_hat = clf.predict(X_validation)
    
# Calculate accuracy 
acc = np.average(y_hat == y_validation)
print('Validation accuracy is:', acc)

Validation accuracy is: 0.9434389140271493


As anticipated, the performance of the model trained on the bottleneck features is much better then the performance of the model trained on raw images. Our approach works. The next step is to find the most optimal classifier. We will accomplish that using the AML feature called `Automated ML`.

## Automated model selection

Automated machine learning (automated ML) automatically picks an algorithm and hyperparameters that optimize a given  primary metric. The model can be downloaded to be further customized as well. There are several options that you can use to configure Automated ML experiments. In this section of the lab we will go through the steps to configure and run the `Automated ML` job.

### Create Experiment

We will create a new experiment to manage `Automated ML' runs

In [6]:
from azureml.core import Experiment

experiment_name = 'aerial-automatedML'
exp = Experiment(workspace=ws, name=experiment_name)

### Create  compute target

Automated ML supports running concurrent experiments on Azure Batch AI clusters. This can significantly shorten the model selection and hyperparameter optimization process. 

**Note:** The creation of the Batch AI cluster can take over 10 minutes, please be patient.


In [7]:
from azureml.core.compute import BatchAiCompute
from azureml.core.compute import ComputeTarget

# Choose a name for your cluster.
batchai_cluster_name = "batchaicls"

found = False
# Check if this compute target already exists in the workspace.
for ct_name, ct in ws.compute_targets().items():
    print(ct.name, ct.type)
    if (ct.name == batchai_cluster_name and ct.type == 'BatchAI'):
        found = True
        print('Found existing compute target.')
        bai_compute_target = ct
        break
        
if not found:
    print('Creating a new compute target...')
    provisioning_config = BatchAiCompute.provisioning_configuration(vm_size = "STANDARD_DS2_V2", 
                                                                autoscale_enabled = True,
                                                                cluster_min_nodes = 1, 
                                                                cluster_max_nodes = 5)

    # Create the cluster.
    bai_compute_target = ComputeTarget.create(ws, batchai_cluster_name, provisioning_config)
    
    # Can poll for a minimum number of nodes and for a specific timeout.
    # If no min_node_count is provided, it will use the scale settings for the cluster.
    bai_compute_target.wait_for_completion(show_output = True, min_node_count = None, timeout_in_minutes = 20)
    
     # For a more detailed view of current Batch AI cluster status, use the 'status' property.

Creating a new compute target...
Creating
succeeded............
BatchAI wait for completion finished
Minimum number of nodes requested have been provisioned


### Create Get Data script

If you are using a remote compute to run your Automated ML experiments, the data fetch must be wrapped in a separate python script that implements `get_data()` function. This script is run on the remote compute where the automated ML experiment is run. `get_data()` eliminates the need to fetch the data over the wire for each iteration. 

In [15]:
import os
script_folder = './script'
script_name = 'get_data.py'
os.makedirs(script_folder, exist_ok=True)

In [16]:
%%writefile $script_folder/get_data.py
import h5py
import numpy as np
import os
from sklearn.model_selection import train_test_split

def get_data():
    # Load bottleneck features
    data_folder = os.environ["AZUREML_DATAREFERENCE_workspacefilestore"]
    file_name = os.path.join(data_folder, 'aerial_bottleneck_resnet50.h5')
    
    print("Data folder:", data_folder)
    print("Bottleneck features file:", file_name)
    print("Data folder content:", os.listdir(data_folder))
    
    with h5py.File(file_name, "r") as hfile:
        features = np.array(hfile.get('features'))
        labels = np.array(hfile.get('labels'))
        
    # Split the data into training and validation partitions   
    X_train, X_validation, y_train, y_validation = train_test_split(features, labels,
                                                               test_size=0.1,
                                                               shuffle=True,
                                                               stratify=labels)
        

    return {'X': X_train, 'y': y_train, 'X_valid': X_validation, 'y_valid': y_validation}

Overwriting ./script/get_data.py


### Configure datastore and data reference

The bottleneck files have been uploaded to the workspace's default datastore during the previous step. We will download the files onto the nodes of the cluster.

In [10]:
from azureml.core import Datastore
from azureml.core.runconfig import DataReferenceConfiguration

ds = ws.get_default_datastore()
print("Using the default datastore for training data: ")
print(ds.name, ds.datastore_type, ds.account_name, ds.container_name)

dr = DataReferenceConfiguration(datastore_name=ds.name, 
                   path_on_datastore='bottleneck_features', 
                   path_on_compute='bottleneck_features',
                   mode='download', # download files from datastore to compute target
                   overwrite=True)


Using the default datastore for training data: 
workspacefilestore AzureFile jkamllab3843747506 azureml-filestore-cc69395a-fa45-471f-8ffb-15ef16d34bda


### Create Docker run configuration

We will run `Automated ML` jobs in a custom docker image that will include dependencies required by `get_data()` script.


In [11]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Run
from azureml.core import ScriptRunConfig

# create a new RunConfig object
run_config = RunConfiguration(framework="python")

# Azure Batch AI cluster for Automated ML jobs require docker.
run_config.environment.docker.enabled = True

# Set compute target to BAI cluster
run_config.target = bai_compute_target.name

# Set data references
run_config.data_references = {ds.name: dr}

# specify packages required by get_data
run_config.environment.python.conda_dependencies = \
   CondaDependencies.create(conda_packages=['h5py'])


### Configure Automated ML run.

Automated ML runs can be controlled using a number of configuration parameters. 


|Property|Description|
|-|-|
|**task**|classification or regression|
|**primary_metric**|This is the metric that you want to optimize.<br> Classification supports the following primary metrics <br><i>accuracy</i><br><i>AUC_weighted</i><br><i>balanced_accuracy</i><br><i>average_precision_score_weighted</i><br><i>precision_score_weighted</i>|
|**max_time_sec**|Time limit in seconds for each iteration|
|**iterations**|Number of iterations. In each iteration Auto ML trains a specific pipeline with the data|
|**n_cross_validations**|Number of cross validation splits|
|**concurrent_iterations**|Max number of iterations that would be executed in parallel. |
|**preprocess**| *True/False* <br>Setting this to *True* enables Auto ML to perform preprocessing <br>on the input to handle *missing data*, and perform some common *feature extraction*|
|**max_cores_per_iteration**| Indicates how many cores on the compute target would be used to train a single pipeline.<br> Default is *1*, you can set it to *-1* to use all cores|
|**exit_score**|*double* value indicating the target for *primary_metric*. <br>Once the target is surpassed the run terminates.|
|**blacklist_algos**|*List* of *strings* indicating machine learning algorithms for AutoML to avoid in this run.<br><br> Allowed values for **Classification**<br><i>LogisticRegression</i><br><i>SGDClassifierWrapper</i><br><i>NBWrapper</i><br><i>BernoulliNB</i><br><i>SVCWrapper</i><br><i>LinearSVMWrapper</i><br><i>KNeighborsClassifier</i><br><i>GradientBoostingClassifier</i><br><i>DecisionTreeClassifier</i><br><i>RandomForestClassifier</i><br><i>ExtraTreesClassifier</i><br><i>LightGBMClassifier</i><br><br>Allowed values for **Regression**<br><i>ElasticNet<i><br><i>GradientBoostingRegressor<i><br><i>DecisionTreeRegressor<i><br><i>KNeighborsRegressor<i><br><i>LassoLars<i><br><i>SGDRegressor<i><br><i>RandomForestRegressor<i><br><i>ExtraTreesRegressor<i>|
|**path**|Relative path to the project folder. AutoML stores configuration files for the experiment under this folder. You can specify a new empty folder.|
    
For the optimal performance of `AutomatedML` it is recommended to run at least 100 iterations. Due to the lab's time constraints we will only run 25 iterations. We will also limit a number of alogirthms tried using the `blacklist_algos` parameter.

In [20]:
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun
import logging


automl_config = AutoMLConfig(run_configuration = run_config,
                             task = 'classification',
                             num_classes = 6,
                             debug_log = 'automl_errors.log',
                             primary_metric = 'accuracy',
                             max_time_sec = 1800,
                             iterations = 25,
                             concurrent_iterations = 5,
                             max_cores_per_iteration = 1,
                             preprocess = False,
                             exit_score = 0.96,
                             blacklist_algos = ['KNeighborsClassifier',
                                                'LinearSVMWrapper',
                                                'NBWrapper',
                                                'BernoulliNB',
                                                'SGDClassifierWrapper'],
                             verbosity = logging.INFO,
                             path = script_folder,
                             data_script = os.path.join(script_folder, script_name))

### Run Automated ML job

In [21]:
tags = {"Desc": "automated ml"}
run = exp.submit(config=automl_config, tags=tags)
run

Experiment,Id,Type,Status,Details Page,Docs Page
aerial-automatedML,AutoML_c9c2c189-9894-4581-8615-640fe2be6543,automl,Preparing,Link to Azure Portal,Link to Documentation


The call to experiment returns `AutoMLRun` object that can be used to track the run.

Since the call is asynchronous, it reports a **Preparing** or **Running** state as soon as the job is started.

Here is what's happening while you wait:

- **Image creation**: A Docker image is created matching the Python environment specified by the RunConfiguration. The image is uploaded to the workspace. This happens only once for each Python environment since the container is cached for subsequent runs.  During image creation, logs are streamed to the run history. You can monitor the image creation progress using these logs.

- **Scaling**: If the remote cluster requires more nodes to execute the run than currently available, additional nodes are added automatically. 

- **Running**: In this stage, the Automated ML takes over and starts running experiments



You can check the progress of a running job in multiple ways: Azure Portal, AML Widgets or streaming logs.

#### Monitor the run.

We will use AML Widget to monitor the run. The widget will first report a "loading" status while running the first iteration. After completing the first iteration, an auto-updating graph and table will be shown. The widget will refresh once per minute, so you should see the graph update as child runs complete.

**Note:** The widget displays a link at the bottom. Use this link to open a web interface to explore the individual run details.

In [22]:
from azureml.train.widgets import RunDetails
RunDetails(run).show()

_AutoML(widget_settings={'childWidgetDisplay': 'popup'})

#### Waiting until the run finishes

`wait_for_complettion` method will block till the run finishes. 

In [ ]:
# Wait until the run finishes.
# run.wait_for_completion(show_output = True)

#### Cancelling Runs

You can cancel ongoing remote runs using the `cancel` and `cancel_iteration` functions.

In [ ]:
# Cancel the ongoing experiment and stop scheduling new iterations.
# run.cancel()

# Cancel iteration 1 and move onto iteration 2.
# run.cancel_iteration(1)

#### Analyze the run

You can  use SDK methods to fetch all the child runs and see individual metrics that we log.

In [ ]:
import pandas as pd

children = list(run.get_children())
metricslist = {}
for child in children:
    properties = child.get_properties()
    metrics = {k: v for k, v in child.get_metrics().items() if isinstance(v, float)}
    metricslist[int(properties['iteration'])] = metrics

rundata = pd.DataFrame(metricslist).sort_index(1)
rundata

### Retrieve the best model

Below we select the best pipeline from our iterations. The `get_output` method returns the best run and the fitted model. The Model includes the pipeline and any pre-processing.  Overloads on `get_output` allow you to retrieve the best run and fitted model for *any* logged metric or for a particular *iteration*.

#### Best model based on the primary metric

In [ ]:
best_run, fitted_model = run.get_output()
print(best_run)
print(fitted_model)

#### Best model based on any metric
You can also retrieve the best model based on an arbitrary metric. For example, we will retrieve the model with the highest AUC value:

In [ ]:
lookup_metric = "AUC_weighted"
best_run_auc, fitted_model_best_auc = run.get_output(metric = lookup_metric)
print(best_run_auc)
print(fitted_model_best_auc)

#### Model from a specific iteration
Show the run and the model from the third iteration:

In [ ]:
iteration = 3
third_run, third_model = run.get_output(iteration=iteration)
print(third_run)
print(third_model)

### Register the best model

You can use `AutoMLRun` to register the trained model with AML Model Registry.

If neither `metric` nor `iteration` are specified in the `register_model` call, the iteration with the best primary metric is registered.

In [ ]:
description = 'Aerial Classifier Model'

tags = {"Featurizer": "ResNet50" }
model = run.register_model(description = description, tags = tags)

print("Model ID:", run.model_id)
print("Run ID:", run.id)

Save *Model ID* and *Run ID*. You will need these values in the deployment lab.

## Next Step

The model is ready for deployment. In the next lab you will deploy the model to Azure Container Instance.

## Clean up resources

Before you move to the next step, you can delete the BAI cluster. We will not need it anymore.

In [ ]:
bai_compute_target.delete()